* Numpy as linear algebra,
* Pandas as data processing,
* PuLP for solving linear programming and optimization problems, including linear and integer programming, mixed-integer programming.

In [1]:
import pandas as pd
from pulp import *

Input data files are available in the read-only _"repositories/Supply chain logistics problem.xlsx"_ directory
**For example**, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [2]:
freight = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'FreightRates')
order = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'OrderList')
plant_ports = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'PlantPorts')
prod_per_plant = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'ProductsPerPlant')
vmi_cust = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'VmiCustomers')
wh_cap = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'WhCapacities')
wh_costs = pd.read_excel('repositories/Supply chain logistics problem.xlsx', sheet_name = 'WhCosts')

In [7]:
wh_cap.head()

,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138


**This code is to create a dictionary that maps origin port codes to destination port codes.**
_In other words, it takes two columns between 'orig_port_cd' and 'dest_port_cs' from a freight DataFrame, where one column contains origin port codes and the other contains destination port codes, then generates a dictionary where each origin port code is mapped to its corresponding destination port code._

In [72]:
dict(zip(freight['orig_port_cd'], freight['dest_port_cd']))

{'PORT08': 'PORT09',
 'PORT10': 'PORT09',
 'PORT09': 'PORT09',
 'PORT11': 'PORT09',
 'PORT04': 'PORT09',
 'PORT02': 'PORT09',
 'PORT03': 'PORT09',
 'PORT07': 'PORT09',
 'PORT05': 'PORT09',
 'PORT06': 'PORT09'}

**We checks for missing values (null values) in each column of the DataFrame freight and returns the sum of null values for each column.
It's essentially a quick way to assess the completeness of data in each column of the DataFrame.**

In [73]:
freight.isnull().sum()

Carrier         0
orig_port_cd    0
dest_port_cd    0
minm_wgh_qty    0
max_wgh_qty     0
svc_cd          0
minimum cost    0
rate            0
mode_dsc        0
tpt_day_cnt     0
Carrier type    0
dtype: int64

**Serves the same purpose as the previous one, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the order DataFrame and returns the sum of null values for each column.**

In [74]:
order.isnull().sum()

Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
dtype: int64

**Serves the same purpose as the previous, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the plant_ports DataFrame and returns the sum of null values for each column.**

In [75]:
plant_ports.isnull().sum()

Plant Code    0
Port          0
dtype: int64

**Serves the same purpose as the previous, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the prod_per_plant DataFrame and returns the sum of null values for each column.**

In [76]:
prod_per_plant.isnull().sum()


Plant Code    0
Product ID    0
dtype: int64

**Serves the same purpose as the previous, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the vmi_cust DataFrame and returns the sum of null values for each column.**

In [77]:
vmi_cust.isnull().sum()

Plant Code    0
Customers     0
dtype: int64

**Serves the same purpose as the previous, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the wh_cap DataFrame and returns the sum of null values for each column.**

In [78]:
wh_cap.isnull().sum()

Plant ID           0
Daily Capacity     0
dtype: int64

**Serves the same purpose as the previous, but it's applied to a DataFrame named order instead of freight. 
It checks for missing values in each column of the wh_costs DataFrame and returns the sum of null values for each column.**

In [79]:
wh_costs.isnull().sum()

WH           0
Cost/unit    0
dtype: int64

Column Selection:
>It selects specific columns from the DataFrame order, including 'Order Date', 'Order ID', 'Carrier', 'Origin Port', 'Destination Port', 'Unit quantity', and 'Weight'. The selected columns are assigned back to the variable order.

Merging DataFrames: 
>It merges the order DataFrame with another DataFrame named plant_ports based on the 'Origin Port' column. The resulting merged DataFrame is assigned to df_merge.

Column Renaming:
>It renames the column 'Plant Code' in df_merge to 'origin_plant_code'.

Column Dropping:
>It drops the 'Port' column from df_merge.

Second Merging:
>It merges df_merge with plant_ports again, this time based on the 'Destination Port' column.

Column Dropping (again):
> It drops the 'Port' column from df_merge again. 

Column Renaming (again):
> It renames the column 'Plant Code' in df_merge to 'destination_plant_code'.

In summary, this code prepares and organizes data by selecting specific columns, merging data from different DataFrames, renaming columns, and dropping unnecessary columns, ultimately creating a DataFrame df_merge that likely includes information about orders, carriers, origin and destination ports, quantities, weights, and corresponding plant codes for both origin and destination.

In [80]:
order = order[['Order Date','Order ID','Carrier','Origin Port','Destination Port','Unit quantity','Weight']]

df_merge = pd.merge(order, plant_ports, left_on = ['Origin Port'], right_on = ['Port'])
df_merge = df_merge.rename(columns = {'Plant Code' : 'origin_plant_code'})
df_merge = df_merge.drop(['Port'], axis = 1)
df_merge = pd.merge(df_merge, plant_ports, left_on = ['Destination Port'], right_on = ['Port'])
df_merge = df_merge.drop(['Port'], axis = 1)
df_merge = df_merge.rename(columns = {'Plant Code' : 'destination_plant_code'})
df_merge.head()

,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,origin_plant_code,destination_plant_code
0,2013-05-26,1.447296e+09,V44_3,PORT09,PORT09,808,14.30,PLANT16,PLANT16
1,2013-05-26,1.447158e+09,V44_3,PORT09,PORT09,3188,87.94,PLANT16,PLANT16
2,2013-05-26,1.447139e+09,V44_3,PORT09,PORT09,2331,61.20,PLANT16,PLANT16
3,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,847,16.16,PLANT16,PLANT16
4,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,2163,52.34,PLANT16,PLANT16


Grouping and Aggregating Data: 
>It groups the DataFrame freight by three columns: 'Carrier', 'orig_port_cd', and 'dest_port_cd'. Then, it aggregates the data within each group using custom aggregation functions defined using lambda functions. Specifically:
- 'minm_wgh_qty'
- 'max_wgh_qty'
- 'rate'
- 'minimum cost' columns
>It calculates the average value within each group.
Resetting Index: After the aggregation, it resets the index of the resulting DataFrame.
Column Renaming: It renames the columns 'minimum cost' and 'minm_wgh_qty' to 'min_cost' and 'min_wgh_qty', respectively.
Displaying the Result: Finally, it displays the resulting DataFrame freight_new, which contains aggregated information about carriers, origin ports, destination ports, and the average values of 'minm_wgh_qty', 'max_wgh_qty', 'rate', and 'minimum cost' for each group.

In [81]:
freight_new = (freight.groupby(['Carrier',
                               'orig_port_cd',
                               'dest_port_cd']).agg({'minm_wgh_qty': lambda x: sum(x)/len(x),
                                                     'max_wgh_qty': lambda x: sum(x)/len(x),
                                                     'rate': lambda x: sum(x)/len(x),
                                                     'minimum cost': lambda x: sum(x)/len(x)}).reset_index()
               .rename(columns = {'minimum cost': 'min_cost', 'minm_wgh_qty': 'min_wgh_qty'}))

freight_new

,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000000,20569.990000,0.084480,23.838400
1,V444_0,PORT03,PORT09,0.000000,5000.000000,12.694200,28.944200
2,V444_0,PORT04,PORT09,570.000000,20569.990000,0.052000,2.477200
3,V444_0,PORT09,PORT09,0.000000,5000.000000,12.778400,31.278400
4,V444_1,PORT02,PORT09,197.556000,1190.745000,0.291140,23.557340
5,V444_1,PORT04,PORT09,197.556000,1190.745000,0.242000,20.909780
6,V444_1,PORT05,PORT09,170.130000,1001.952500,0.457895,35.954091
7,V444_1,PORT06,PORT09,45.473493,111.272697,2.492247,6.917300
8,V444_1,PORT10,PORT09,78.945000,12570.433750,0.431400,12.350850
9,V444_2,PORT02,PORT09,570.000000,20569.990000,0.079040,9.027200


Generate Cost Between Origin and Destination
>From PORT02 to PORT11 in chronological order w/ the same destination to PORT09

In [82]:
cost = freight_new.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
cost = round(cost,0)
cost = cost[cost.index.isin(df_merge['Origin Port'])]
cost_pulp = cost.to_numpy().tolist()
cost_pulp

[[2.0], [8.0], [24.0]]

**Check the cost**

In [83]:
cost

dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT05,8.0
PORT09,24.0


**Check the wh_cap**

In [84]:
wh_cap

,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138
5,PLANT01,1070
6,PLANT06,49
7,PLANT10,118
8,PLANT07,265
9,PLANT14,549


Creating Lists of Unique Ports:
>It extracts unique values from the 'Origin Port' and 'Destination Port' columns of the DataFrame df_merge and converts them into sorted lists, storing them in variables origins and destinations, respectively.

Aggregating Data for Origins and Destinations:
>It groups the df_merge DataFrame separately by 'Origin Port' and 'Destination Port', aggregating several metrics:

   - Number of unique order IDs ('Order ID')
   - Number of unique order dates ('Order Date')
   - Total unit quantity ('Unit quantity')
   - Total weight ('Weight')

Calculating Averages:
>It calculates the average quantity per order ('avg_qty') and the average weight per order ('avg_weight') for both origin and destination ports.

Dropping Redundant Columns:
> It drops the 'Order ID', 'Unit quantity', and 'Weight' columns from both df_origin and df_dest DataFrames, keeping only the aggregated metrics and calculated averages.

Displaying the Result for Destinations:
>Finally, it displays the resulting DataFrame df_dest, which contains aggregated information about unique order IDs, unique order dates, total unit quantity, total weight, and calculated average quantity and weight per order for each destination port.

In [85]:
origins = df_merge['Origin Port'].unique().tolist()
origins = sorted(origins)
destinations = df_merge['Destination Port'].unique().tolist()
df_origin = df_merge.groupby(['Origin Port']).agg({'Order ID': 'nunique',
                                                   'Order Date': 'nunique',
                                                   'Unit quantity': 'sum',
                                                   'Weight': 'sum'}).reset_index()
df_origin['avg_qty'] = round(df_origin['Unit quantity']/df_origin['Order Date'], 0)
df_origin['avg_weight'] = round(df_origin['Weight']/df_origin['Order Date'], 0)
df_origin = df_origin.drop(['Order ID', 'Unit quantity', 'Weight'], axis = 1)
df_dest = df_merge.groupby(['Destination Port']).agg({'Order ID': 'nunique',
                                                      'Order Date': 'nunique',
                                                      'Unit quantity': 'sum',
                                                      'Weight' : 'sum'}).reset_index()
df_dest['avg_qty'] = round(df_dest['Unit quantity']/df_dest['Order Date'], 0)
df_dest['avg_weight'] = round(df_dest['Weight']/df_dest['Order Date'], 0)
df_dest = df_dest.drop(['Order ID', 'Unit quantity', 'Weight'], axis = 1)

df_dest

,Destination Port,Order Date,avg_qty,avg_weight
0,PORT09,1,205120319.0,1088803.0


**Now check df_origin DataFrame**

In [86]:
df_origin

,Origin Port,Order Date,avg_qty,avg_weight
0,PORT04,1,204874838.0,1056632.0
1,PORT05,1,348.0,2.0
2,PORT09,1,245133.0,32168.0


Creating Supply and Demand Dictionaries:
>It creates dictionaries supply and demand using the zip function, where keys are port codes and values are average weights. These dictionaries are based on aggregated data from df_origin and df_dest DataFrames, representing the average weight of orders for each origin and destination port, respectively.
Displaying Information:
> It prints out various pieces of information:
   - All unique origin nodes (origins).
   - Supply information, which maps each origin port to its average weight (supply).
   - All unique destination nodes (destinations).
   - Demand information, which maps each destination port to its average weight (demand).
   - Costs for routing optimization (cost_pulp). Note: cost_pulp seems to be a variable not defined in this code snippet.

Optional:
>There are commented lines that can be uncommented to manually assign supply values for certain ports and demand values for a specific port.

In [87]:
supply = dict(zip(df_origin['Origin Port'],df_origin['avg_weight']))
demand = dict(zip(df_dest['Destination Port'], df_dest['avg_weight']))

#All Variables for Routing Optimization 
# supply['PORT04'] = 1013
# supply['PORT05'] = 385
# supply['PORT09'] = 457
# demand['PORT09'] = 457

print(f'All Origin Nodes : {origins}')
print(f'Origin Supply : {supply}')
print(f'All Destination Nodes : {destinations}')
print(f'Destination Demand : {demand}')
print(f'Costs From Origin to Destination : {cost_pulp}')

All Origin Nodes : ['PORT04', 'PORT05', 'PORT09']
Origin Supply : {'PORT04': 1056632.0, 'PORT05': 2.0, 'PORT09': 32168.0}
All Destination Nodes : ['PORT09']
Destination Demand : {'PORT09': 1088803.0}
Costs From Origin to Destination : [[2.0], [8.0], [24.0]]


```costs = makeDict([origins,destinations], cost_pulp, 0)```:
>This line creates a dictionary costs where keys are tuples of origin and destination ports, and values are the associated transportation costs. The function makeDict is likely a custom function defined elsewhere to create a dictionary from given inputs.

```prob = LpProblem("Port Distribution Problem", LpMinimize)```: 
>This line creates a new linear programming problem named "Port Distribution Problem" with the objective of minimizing costs (LpMinimize).

```routes = [(o,d) for o in origins for d in destinations]```:
>This line generates all possible combinations of routes between origin and destination ports.

```vars = LpVariable.dicts("route", (origins,destinations), 0 , None, LpInteger)```:
>This line creates a dictionary vars of decision variables representing the amount of goods to be transported from each origin to each destination. The decision variables are indexed by origin and destination ports and are constrained to be integers greater than or equal to 0.

```prob += (lpSum([vars[o][d] * costs[o][d] for (o,d) in routes]), "Sum of Transporting Costs")```:
>This line adds the objective function to the problem. The objective function is the sum of transportation costs for all routes, calculated as the product of the decision variable representing the amount of goods to be transported on each route and the associated transportation cost.

```prob```:
>The prob variable represents the linear programming problem that has been set up, including the objective function and any constraints that may be added later.

In [88]:
costs = makeDict([origins,destinations], cost_pulp, 0)
prob = LpProblem("Port Distribution Problem", LpMinimize)
routes = [(o, d) for o in origins for d in destinations]
varying = LpVariable.dicts("route", (origins,destinations), 0, None, LpInteger)
prob += (lpSum([varying[o][d] * costs[o][d] for (o, d) in routes]), "Sum of Transporting Costs")

prob

C:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\extensorflowgraphql-1Hc8Reqa-py3.11\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Port_Distribution_Problem:
MINIMIZE
2.0*route_PORT04_PORT09 + 8.0*route_PORT05_PORT09 + 24.0*route_PORT09_PORT09 + 0.0
VARIABLES
0 <= route_PORT04_PORT09 Integer
0 <= route_PORT05_PORT09 Integer
0 <= route_PORT09_PORT09 Integer

**Supply Max Constraints from each Origins Node**

In [89]:
for o in origins :
    prob += (lpSum([varying[o][d] for d in destinations]) <= supply[o],
             "Sum_of_Weight_of_Goods_out_of_Origin_Ports_%s" % o)
    
for d in destinations :
    prob += (lpSum([varying[o][d] for o in origins]) >= demand[d],
             "Sum_of_Weight_of_Goods_into_Destination_Ports_%s" % d)

```prob.solve()```:   
>This line solves the linear programming problem prob using PuLP's default solver. The solver computes the optimal solution to minimize the objective function while satisfying any constraints.

```for v in prob.variables()```:
>This iterates over all decision variables defined in the problem.

```print(f'{v}: {v.varValue}')```:
>For each decision variable v, this line prints out its name and the optimal value found by the solver (v.varValue). The optimal value represents the amount of goods to be transported from the corresponding origin port to the corresponding destination port.

In [90]:
prob.solve()

for v in prob.variables():
    print(f'{v}: {v.varValue}')

route_PORT04_PORT09: 1056632.0
route_PORT05_PORT09: 2.0
route_PORT09_PORT09: 32169.0


Grouping and Aggregating Data:
>It groups the DataFrame freight by pairs of origin and destination port codes (orig_port_cd and dest_port_cd), and then calculates the average of the 'minimum cost' within each group, rounding it to the nearest integer.

Reshaping Data:
>It reshapes the resulting aggregated data into a pivot table using the unstack() function, where the rows correspond to origin port codes and the columns correspond to destination port codes.

Converting to Numpy Array:
>It converts the 'minimum cost' values from the pivot table into a Numpy array and then converts the array into a list.

In [91]:
cost_2 = freight.groupby(['orig_port_cd', 
                          'dest_port_cd']).agg({'minimum cost': lambda x: round((x.max() + x.min())/2, 0)}).unstack()
cost_pulp_2 = cost_2['minimum cost'].to_numpy().tolist()

cost_pulp_2

[[60.0],
 [222.0],
 [52.0],
 [107.0],
 [60.0],
 [16.0],
 [125.0],
 [24.0],
 [33.0],
 [18.0]]

```plant_ports_filtered = plant_ports.set_index('Port')```:
>This line sets the index of the DataFrame plant_ports to the 'Port' column, creating a new DataFrame plant_ports_filtered with the 'Port' column as its index.

```plant_ports_filtered = plant_ports_filtered[plant_ports_filtered.index.isin(cost_2.index)]```:
>This line filters plant_ports_filtered to include only rows where the index values (port codes) are present in the index of the DataFrame cost_2. In other words, it keeps only those rows from plant_ports_filtered that correspond to ports for which cost data is available.

```pp_filtered = plant_ports_filtered.reset_index().sort_values(by = 'Port')```:
>This line resets the index of plant_ports_filtered and sorts the DataFrame by the 'Port' column in ascending order. It assigns the result to a new DataFrame pp_filtered. This step is done to revert the index back to a regular column and ensure that the DataFrame is sorted by port codes for better reada bility or further analysis.

In [92]:
plant_ports_filtered = plant_ports.set_index('Port')
plant_ports_filtered = plant_ports_filtered[plant_ports_filtered.index.isin(cost_2.index)]
pp_filtered = plant_ports_filtered.reset_index().sort_values(by = 'Port')

pp_filtered

,Port,Plant Code
0,PORT02,PLANT01
6,PORT02,PLANT07
9,PORT02,PLANT10
1,PORT03,PLANT02
7,PORT04,PLANT08
8,PORT04,PLANT09
10,PORT04,PLANT11
11,PORT04,PLANT12
12,PORT04,PLANT13
2,PORT04,PLANT03


```pp_merged = pd.merge(pp_filtered, wh_cap, left_on=['Plant Code'], right_on=['Plant ID'], how='left')```:
>This line merges the pp_filtered DataFrame (which likely contains information about plant ports) with the wh_cap DataFrame (which presumably contains warehouse capacity information), based on the 'Plant Code' column in pp_filtered and the 'Plant ID' column in wh_cap. The merge is performed using a left join (how='left'), meaning all rows from pp_filtered are retained, and matching rows from wh_cap are included if they exist. The result is stored in pp_merged.

```pp_merged = pp_merged.drop(['Plant ID'], axis=1)```:
>This line drops the 'Plant ID' column from the pp_merged DataFrame, as it may have been duplicated after the merge.

```pp_sum = pp_merged.groupby(['Port'])['Daily Capacity '].sum().reset_index()```:
>This line groups the pp_merged DataFrame by the 'Port' column and calculates the sum of the 'Daily Capacity' for each port. The result is stored in pp_sum, which likely contains the total daily capacity for each port. The reset_index() function is used to convert the grouped result back into a DataFrame with a default integer index.

In [93]:
pp_merged = pd.merge(pp_filtered, wh_cap, left_on = ['Plant Code'], right_on = ['Plant ID'], how = 'left')
pp_merged = pp_merged.drop(['Plant ID'], axis = 1)
pp_sum = pp_merged.groupby(['Port'])['Daily Capacity '].sum().reset_index()

pp_sum

,Port,Daily Capacity
0,PORT02,1453
1,PORT03,138
2,PORT04,2076
3,PORT05,554
4,PORT06,434
5,PORT07,549
6,PORT08,11
7,PORT09,457
8,PORT10,8
9,PORT11,111


```perc_needed = pp_sum[pp_sum['Port'] == 'PORT09']['Daily Capacity ']```:
>his line filters the pp_sum DataFrame to include only rows where the 'Port' column equals 'PORT09', and then selects the 'Daily Capacity' column from the filtered rows. It stores the result in perc_needed, which likely contains the total daily capacity for the port 'PORT09'.

```sum(pp_sum['Daily Capacity'])```:
>This part calculates the total daily capacity across all ports by summing the 'Daily Capacity' column of the pp_sum DataFrame.

```perc_needed = perc_needed / sum(pp_sum['Daily Capacity'])```:
>This line divides the total daily capacity needed for 'PORT09' by the total daily capacity across all ports. The result is the percentage of the total daily capacity that is needed for 'PORT09'.

In [94]:
perc_needed = pp_sum[pp_sum['Port'] == 'PORT09']['Daily Capacity ']/sum(pp_sum['Daily Capacity '])

perc_needed

7    0.078916
Name: Daily Capacity , dtype: float64

**Check columns on pp_merged**

In [95]:
pp_merged.columns

Index(['Port', 'Plant Code', 'Daily Capacity '], dtype='object')

```f_origin = freight[['orig_port_cd']].drop_duplicates().sort_values(by='orig_port_cd')```:
>This line extracts unique origin port codes from the 'orig_port_cd' column of the freight DataFrame, removes any duplicate values, and sorts them in ascending order.

```f_dest = freight[['dest_port_cd']].drop_duplicates().sort_values(by='dest_port_cd')```:
>This line performs a similar operation for destination ports, extracting unique destination port codes from the 'dest_port_cd' column of the freight DataFrame, removing duplicates, and sorting them in ascending order.

```f_origin = pd.merge(f_origin, pp_sum, left_on='orig_port_cd', right_on='Port', how='left')```:
>This line merges the DataFrame f_origin with the pp_sum DataFrame based on the 'orig_port_cd' column in f_origin and the 'Port' column in pp_sum. The merge is performed using a left join (how='left'), ensuring that all rows from f_origin are retained. The result contains the daily capacity information for each origin port.

```f_origin = f_origin[['orig_port_cd','Daily Capacity ']]```:
>This line selects only the 'orig_port_cd' and 'Daily Capacity' columns from the merged DataFrame f_origin, discarding any other columns.

```f_origin_list = f_origin['orig_port_cd'].tolist()```:
>Finally, this line converts the 'orig_port_cd' column of the DataFrame f_origin into a list named f_origin_list, which likely contains the unique origin port codes.

In [96]:
f_origin = freight[['orig_port_cd']].drop_duplicates().sort_values(by = 'orig_port_cd')
f_dest = freight[['dest_port_cd']].drop_duplicates().sort_values(by = 'dest_port_cd')

f_origin = pd.merge(f_origin, pp_sum, left_on = 'orig_port_cd', right_on = 'Port', how = 'left')
f_origin = f_origin[['orig_port_cd','Daily Capacity ']]

f_origin_list = f_origin['orig_port_cd'].tolist()
f_origin_list

['PORT02',
 'PORT03',
 'PORT04',
 'PORT05',
 'PORT06',
 'PORT07',
 'PORT08',
 'PORT09',
 'PORT10',
 'PORT11']

```f_dest = pd.merge(f_dest, pp_sum, left_on='dest_port_cd', right_on='Port', how='left')```:
>This line merges the DataFrame f_dest with the pp_sum DataFrame based on the 'dest_port_cd' column in f_dest and the 'Port' column in pp_sum. The merge is performed using a left join (how='left'), ensuring that all rows from f_dest are retained. The result contains the daily capacity information for each destination port.

```f_dest = f_dest[['dest_port_cd','Daily Capacity ']]```:
>This line selects only the 'dest_port_cd' and 'Daily Capacity' columns from the merged DataFrame f_dest, discarding any other columns.

```f_dest_list = f_dest['dest_port_cd'].tolist()```:
>Finally, this line converts the 'dest_port_cd' column of the DataFrame f_dest into a list named f_dest_list, which likely contains the unique destination port codes.

In [97]:
f_dest = pd.merge(f_dest, pp_sum, left_on = 'dest_port_cd', right_on = 'Port', how = 'left')
f_dest = f_dest[['dest_port_cd','Daily Capacity ']]
f_dest

f_dest_list = f_dest['dest_port_cd'].tolist()
f_dest_list

['PORT09']

```origin_cap = dict(zip(f_origin['orig_port_cd'], f_origin['Daily Capacity '].astype('float')))```:
>This line creates a dictionary origin_cap where keys are origin port codes ('orig_port_cd' column from the DataFrame f_origin) and values are their corresponding daily capacities ('Daily Capacity ' column from the DataFrame f_origin). The astype('float') method is used to convert the values to floating-point numbers.

```dest_cap = dict(zip(f_dest['dest_port_cd'], f_dest['Daily Capacity '].astype('float')))```:
>Similarly, this line creates a dictionary dest_cap where keys are destination port codes ('dest_port_cd' column from the DataFrame f_dest) and values are their corresponding daily capacities ('Daily Capacity ' column from the DataFrame f_dest). Again, the astype('float') method is used to convert the values to floating-point numbers.

```origin_cap```:
>This variable holds the origin_cap dictionary, which maps origin port codes to their daily capacities. When printed, it would display the dictionary containing the mapping between origin port codes and their daily capacities.

In [98]:
origin_cap = dict(zip(f_origin['orig_port_cd'], f_origin['Daily Capacity '].astype('float')))
dest_cap = dict(zip(f_dest['dest_port_cd'], f_dest['Daily Capacity '].astype('float')))

origin_cap

{'PORT02': 1453.0,
 'PORT03': 138.0,
 'PORT04': 2076.0,
 'PORT05': 554.0,
 'PORT06': 434.0,
 'PORT07': 549.0,
 'PORT08': 11.0,
 'PORT09': 457.0,
 'PORT10': 8.0,
 'PORT11': 111.0}

**Check that dest_cap**

In [99]:
dest_cap

{'PORT09': 457.0}

```problem_statement = LpProblem("Port Distribution Problem", LpMinimize)```:
>This line creates a new linear programming problem named "Port Distribution Problem" with the objective of minimizing costs (LpMinimize).

```variables = LpVariable.dicts("route", (f_origin_list, f_dest_list), 0 , None, LpInteger)```:
>This line creates decision variables representing the amount of goods to be transported from each origin port to each destination port. The decision variables are indexed by origin and destination port codes and are constrained to be integers greater than or equal to 0.

```routes = [(o,d) for o in f_origin_list for d in f_dest_list]```:
>This line generates all possible combinations of routes between origin and destination ports.

```costs = makeDict([f_origin_list, f_dest_list], cost_pulp_2, 0)```:
>This line creates a dictionary costs where keys are tuples of origin and destination port codes, and values are the associated transportation costs (cost_pulp_2). This dictionary represents the costs of transporting goods along each route.

```problem_statement```:
>This line adds the objective function to the problem. The objective function is the sum of transportation costs for all routes, calculated as the product of the decision variable representing the amount of goods to be transported on each route and the associated transportation cost.

```the loop```:
>These loops add capacity constraints to the problem. For each origin port, the total weight of goods leaving the port must not exceed the port's capacity (origin_cap). Similarly, for each destination port, the total weight of goods arriving at the port must meet or exceed the port's capacity (dest_cap).

```problem_statement.solve()```:
>This line solves the linear programming problem defined by problem_statement using PuLP's solver.

```last loop```:
>This loop iterates over all decision variables in the problem and prints out their names and optimal values. The optimal value represents the amount of goods to be transported along each route in the optimal solution.

In [100]:
problem_statement = LpProblem("Port Distribution Problem", LpMinimize)
variables = LpVariable.dicts("route", (f_origin_list,f_dest_list), 0, None, LpInteger)
routes = [(o, d) for o in f_origin_list for d in f_dest_list]
costs = makeDict([f_origin_list,f_dest_list], cost_pulp_2, 0)
problem_statement += (lpSum([variables[o][d] * costs[o][d] for (o, d) in routes]),
                      "Sum_of_Transporting_Costs")

for o in f_origin_list :
    problem_statement += (lpSum([variables[o][d] for d in f_dest_list]) <= origin_cap[o],
                          "Sum_of_Weight_out_of_Port_%s" %o)
    
for d in f_dest_list :
    problem_statement += (lpSum([variables[o][d] for o in f_origin_list]) >= dest_cap[d],
                          "Sum_of_Weight_in_Port_%s" %d)
    
problem_statement.solve()    
    
for a in problem_statement.variables():
    print(f'{a}: {a.varValue}')

route_PORT02_PORT09: 0.0
route_PORT03_PORT09: 0.0
route_PORT04_PORT09: 0.0
route_PORT05_PORT09: 0.0
route_PORT06_PORT09: 0.0
route_PORT07_PORT09: 457.0
route_PORT08_PORT09: 0.0
route_PORT09_PORT09: 0.0
route_PORT10_PORT09: 0.0
route_PORT11_PORT09: 0.0


C:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\extensorflowgraphql-1Hc8Reqa-py3.11\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
